In [1]:
import requests
import json
import numpy as np
import pandas as pd

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import rasterio as rt
from rasterio.mask import mask

%matplotlib inline
import matplotlib.pyplot as plt
from shapely.geometry.polygon import Polygon

import plotly.graph_objects as go

from typing import Dict 

In [2]:
address = "Lange Van Sterbeeckstraat 8, 2060 Antwerpen"

In [6]:

def get_address_data_from_geopunt(address: str):
    data = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    print(data,"\n")
    info = {'address' : address, 
                'x_value' : data['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : data['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : data['LocationResult'][0]['Thoroughfarename'],
                'house_number' : data['LocationResult'][0]['Housenumber'], 
                'postcode': data['LocationResult'][0]['Zipcode'], 
                'municipality' : data['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
#    print(info['polygon'][0]['coordinates'][0])
#    print(info['polygon'][1]['coordinates'][1])
    
    
    return info

In [7]:
address_info = get_address_data_from_geopunt(address)
address_info

{'LocationResult': [{'Municipality': 'Antwerpen', 'Zipcode': '2060', 'Thoroughfarename': 'Lange Van Sterbeeckstraat', 'Housenumber': '8', 'ID': 949971, 'FormattedAddress': 'Lange Van Sterbeeckstraat 8, 2060 Antwerpen', 'Location': {'Lat_WGS84': 51.22387124286627, 'Lon_WGS84': 4.424710567489425, 'X_Lambert72': 153908.9, 'Y_Lambert72': 212660.38}, 'LocationType': 'crab_huisnummer_afgeleidVanGebouw', 'BoundingBox': {'LowerLeft': {'Lat_WGS84': 51.22387124286627, 'Lon_WGS84': 4.424710567489425, 'X_Lambert72': 153908.9, 'Y_Lambert72': 212660.38}, 'UpperRight': {'Lat_WGS84': 51.22387124286627, 'Lon_WGS84': 4.424710567489425, 'X_Lambert72': 153908.9, 'Y_Lambert72': 212660.38}}}]} 



{'address': 'Lange Van Sterbeeckstraat 8, 2060 Antwerpen',
 'x_value': 153908.9,
 'y_value': 212660.38,
 'street': 'Lange Van Sterbeeckstraat',
 'house_number': '8',
 'postcode': '2060',
 'municipality': 'Antwerpen',
 'polygon': [{'coordinates': [[[153913.09697395563, 212652.07560559362],
     [153909.20398195833, 212669.40699760243],
     [153904.5820299536, 212668.39899760485],
     [153907.5011979565, 212656.54043759406],
     [153908.57345395535, 212652.1845335923],
     [153908.816653952, 212651.1965655908],
     [153909.12673395127, 212651.2602455914],
     [153913.09697395563, 212652.07560559362]]],
   'type': 'Polygon'}]}

In [9]:
poly = address_info["polygon"]
type(poly)


list

In [11]:
x_arr = []
y_arr = []

pv = poly[0]['coordinates']
 #print(pv)
for i in range(len(pv)):
    #print(pv[i])
    lst = pv[i]
for j in range(len(lst)):
    #print(lst[j])
    x_arr.append(lst[j][0])
    y_arr.append(lst[j][1])
x_arr = np.array(x_arr)
y_arr = np.array(y_arr)
print("x: ", x_arr)
print("y: ", y_arr)  


x:  [153913.09697396 153909.20398196 153904.58202995 153907.50119796
 153908.57345396 153908.81665395 153909.12673395 153913.09697396]
y:  [212652.07560559 212669.4069976  212668.3989976  212656.54043759
 212652.18453359 212651.19656559 212651.26024559 212652.07560559]


SyntaxError: invalid syntax (<ipython-input-25-c2e363b50054>, line 13)

In [23]:
x

array([153894.58202995, 153895.58202995, 153896.58202995, 153897.58202995,
       153898.58202995, 153899.58202995, 153900.58202995, 153901.58202995,
       153902.58202995, 153903.58202995, 153904.58202995, 153905.58202995,
       153906.58202995, 153907.58202995, 153908.58202995, 153909.58202995,
       153910.58202995, 153911.58202995, 153912.58202995, 153913.58202995,
       153914.58202995, 153915.58202995, 153916.58202995, 153917.58202995,
       153918.58202995, 153919.58202995, 153920.58202995, 153921.58202995,
       153922.58202995])

In [24]:
y

array([212641.19656559, 212642.19656559, 212643.19656559, 212644.19656559,
       212645.19656559, 212646.19656559, 212647.19656559, 212648.19656559,
       212649.19656559, 212650.19656559, 212651.19656559, 212652.19656559,
       212653.19656559, 212654.19656559, 212655.19656559, 212656.19656559,
       212657.19656559, 212658.19656559, 212659.19656559, 212660.19656559,
       212661.19656559, 212662.19656559, 212663.19656559, 212664.19656559,
       212665.19656559, 212666.19656559, 212667.19656559, 212668.19656559,
       212669.19656559, 212670.19656559, 212671.19656559, 212672.19656559,
       212673.19656559, 212674.19656559, 212675.19656559, 212676.19656559,
       212677.19656559, 212678.19656559, 212679.19656559])

In [1]:

import pandas as pd

from mpl_toolkits import mplot3d
#from mpl_toolkits import axes3d
import numpy as np
import rasterio
from rasterio.plot import show

import matplotlib.pyplot as plt
%matplotlib inline

# Get DTM data
dtm_src = 'data/DHMVIIDTMRAS1m_k01/GeoTIFF/DHMVIIDTMRAS1m_k01.tif'
dtm_img = rasterio.open(dtm_src)
dtm_array = dtm_img.read()

# Get DSM data
dsm_src = 'data/DHMVIIDSMRAS1m_k01/GeoTIFF/DHMVIIDSMRAS1m_k01.tif'
dsm_img = rasterio.open(dsm_src)
#show(dsm_img)
dsm_array = dsm_img.read()

# Calculate the CHM (Canopy Height Model)
height_dsm_min_dtm = dsm_array - dtm_array
z_arr = np.array(height_dsm_min_dtm)
z_arr = np.array(z_arr)

X, Y = np.meshgrid(x_arr, y_arr)

Z = np.array(z_arr)
#ax.plot_surface(X, Y, z)
print(Z)
#x, y = np.meshgrid(x_arr, y_arr)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, z_arr)

#ax = plt.axes(projection="3d")

#ax.plot.surface(x, y, z)
#ax.set_title("surface plot")
plt.show()


NameError: name 'x_arr' is not defined

In [2]:
print(height_dsm_min_dtm)

[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.02000046]
  [0.         0.         0.         ... 0.01000023 0.         0.01000023]]]


In [73]:
xh_arr = []
yh_arr = []

heights = height_dsm_min_dtm
#print(heights)
for i in range(len(heights)):
    #print(pv[i])
    lst = heights[i]
for j in range(len(lst)):
    #print(lst[j])
    xh_arr.append(lst[j][-2])
    yh_arr.append(lst[j][-1])
 
    
xh_arr = np.array(xh_arr)
yh_arr = np.array(yh_arr)
print(xh_arr)
print(yh_arr) 

[0. 0. 0. ... 0. 0. 0.]
[0.         0.         0.         ... 0.         0.02000046 0.01000023]


(9000,)


9000

(9000,)


9000